# Autheur: JOSHUA JUSTE EMMANUEL YUN PEI NIKIEMA
## Classe: INGC3 InDIA
## Date: Jeudi 11 Avril 2024

# Importation des librairies

In [1]:
import pandas as pd
import numpy as np
import os
import torch  # Import de la bibliothèque PyTorch pour le deep learning
from torch.utils.data import TensorDataset, DataLoader  # Pour les ensembles de données et les chargeurs de données
import numpy as np  # Pour les opérations numériques
from sklearn.pipeline import Pipeline  # Pour créer un pipeline de prétraitement des données
from sklearn.preprocessing import StandardScaler, LabelEncoder  # Pour le prétraitement des données
from sklearn.model_selection import train_test_split  # Pour la séparation des données en ensembles d'entraînement et de test
from sklearn.impute import SimpleImputer  # Pour remplir les valeurs manquantes dans les données
from sklearn.pipeline import Pipeline
import torch.nn.functional as F  # Pour les fonctions d'activation et les fonctions de perte
from torch.utils.tensorboard import SummaryWriter  # Pour la visualisation des résultats avec TensorBoard

2024-04-11 01:24:21.790826: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-11 01:24:21.790927: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-11 01:24:21.933293: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Création de la Classe pour le prétraitement des données

In [2]:
class DataProcessor:
    """Classe pour prétraiter les données."""

    def __init__(self, device):
        """Initialiseur de la classe DataProcessor."""
        self.encoder = LabelEncoder()
        self.preprocessing_pipeline = Pipeline(steps=[
            ('imputer', SimpleImputer()),  # Remplace les valeurs manquantes
            ('scaler', StandardScaler())   # Met à l'échelle les caractéristiques
        ])
        self.device = device

    
    def preprocess_data(self, data, target_name):
        """Prétraite les données en remplissant les valeurs manquantes, en mettant à l'échelle et en encodant la variable cible.

        Args:
            data (pandas.DataFrame): Le DataFrame contenant les données à prétraiter.
            target_name (str): Le nom de la colonne contenant la variable cible.

        Returns:
            torch.Tensor, torch.Tensor: Les données prétraitées (caractéristiques) et la variable cible encodée.
        """

        target = data[target_name]
        data = data.drop(columns=[target_name])
        data = data.replace(to_replace=[np.Inf, -np.Inf], value=np.nan)
        
        # Conversion des colonnes en int ou float
        for col in data.columns:
            try:
                data[col] = pd.to_numeric(data[col], errors='coerce')  # Tente une conversion numérique générale
            except ValueError:
                pass
        
        data = data.replace(to_replace=[np.Inf, -np.Inf], value=np.nan)

        # Conversion en tenseur et transfert sur le périphérique approprié (GPU ou CPU)
        data_tensor = torch.from_numpy(data.values).float().to(self.device)
        target_tensor = torch.from_numpy(self.encoder.fit_transform(target)).long().to(self.device)

        # Application du pipeline de prétraitement
        preprocessed_data = self.preprocessing_pipeline.fit_transform(data_tensor.cpu().numpy())
        preprocessed_data_tensor = torch.from_numpy(preprocessed_data).float().to(self.device)

        return preprocessed_data_tensor, target_tensor


# Création de la Classe pour Gérer les Chargeurs des données

In [3]:
class DataLoaderManager:
    """
    Classe pour gérer la création et la configuration des chargeurs de données pour l'entraînement et l'évaluation de modèles.
    """

    def __init__(self, batch_size_train=100, batch_size_test=20, device=None):
        """
        Initialise la classe DataLoaderManager.

        Args:
            batch_size_train (int, optional): Taille des lots pour l'entraînement. Par défaut, 100.
            batch_size_test (int, optional): Taille des lots pour l'évaluation. Par défaut, 20.
            device (torch.device, optional): Périphérique utilisé pour le calcul (CPU ou GPU). 
                                            Si None, le périphérique sera déterminé automatiquement.
        """

        self.batch_size_train = 100
        self.batch_size_test = 20
        self.device = device # Détermine le périphérique

    def create_datasets_and_loaders(self, X_train, X_test, y_train, y_test):
        """
        Crée des ensembles de données et des chargeurs de données pour l'entraînement et l'évaluation.

        Args:
            X_train (np.ndarray): Données d'entraînement (caractéristiques).
            X_test (np.ndarray): Données de test (caractéristiques).
            y_train (np.ndarray): Étiquettes d'entraînement.
            y_test (np.ndarray): Étiquettes de test.

        Returns:
            tuple: Quatre objets :
                - trainset (TensorDataset): Ensemble de données d'entraînement.
                - testset (TensorDataset): Ensemble de données de test.
                - trainloader (DataLoader): Chargeur de données pour l'entraînement.
                - testloader (DataLoader): Chargeur de données pour l'évaluation.
        """

        # transfert sur le périphérique approprié
        X_train, y_train = X_train.to(self.device), y_train.to(self.device)
        X_test, y_test = X_test.to(self.device), y_test.to(self.device)

        # Création des ensembles de données
        trainset = TensorDataset(X_train, y_train)
        testset = TensorDataset(X_test, y_test)

        # Création des chargeurs de données
        trainloader = DataLoader(dataset=trainset, batch_size=self.batch_size_train, shuffle=True)
        testloader = DataLoader(dataset=testset, batch_size=self.batch_size_test, shuffle=True)

        return trainset, testset, trainloader, testloader


# Création de la Classe de notre Modèle

In [4]:
class IntrusionDetectionModel(torch.nn.Module):
    """Classe pour le modèle de détection d'intrusion."""

    def __init__(self, in_features, out_features, device):
        """Initialiseur de la classe IntrusionDetectionModel."""
        super().__init__()
        self.fc1 = torch.nn.Linear(in_features, 50)
        self.fc2 = torch.nn.Linear(50, 100)
        self.fc3 = torch.nn.Linear(100, out_features)
        self.device = device
        self.to(self.device)

    def forward(self, x):
        """Méthode de propagation avant."""
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# Création d'une fonction pour charger et transformer les données en DataFrame

In [5]:

import psutil

def load_csv_files_progressively(directory, memory_threshold=70):
    """
    Charge tous les fichiers CSV dans un répertoire donné et les combine progressivement en un unique DataFrame.

    Args:
    - directory (str): Chemin du répertoire contenant les fichiers CSV.
    - memory_threshold (int): Seuil d'utilisation de la mémoire RAM en pourcentage pour arrêter la concaténation.

    Returns:
    - DataFrame: DataFrame combinant toutes les données des fichiers CSV progressivement.
    - int: Nombre de DataFrames concaténés.
    """

    # Liste pour stocker les DataFrames combinés progressivement
    combined_dfs = []
    file_to_exclude = ['02-20-2018.csv', '03-01-2018.csv', '02-16-2018.csv']

    # Parcours des fichiers dans le répertoire
    files = [file for file in os.listdir(directory) if file.endswith('.csv') and file not in file_to_exclude]


    # Surveillance de l'utilisation de la mémoire
    mem = psutil.virtual_memory()

    # Lecture et concaténation progressive des fichiers
    for i, file in enumerate(files):
        print('Fichier utilisé : ', file)
        file_path = os.path.join(directory, file)
        df = pd.read_csv(file_path)
        df = df.drop(columns=["Timestamp"], axis=1)

        if i == 0:
            combined_df = df
        else:
            combined_df = pd.concat([combined_df, df], ignore_index=True)


        # Vérifier l'utilisation de la mémoire
        mem = psutil.virtual_memory()
        if mem.percent >= memory_threshold:
            break

    # Retourner le DataFrame combiné progressivement et le nombre de DataFrames concaténés
    return combined_df, i + 1


# Création de la fonction de la boucle d'entrainement

In [6]:
def train_model(model, criterion, optimizer, trainloader, max_iter, writer, device):
    """
    Fonction pour entraîner le modèle et enregistrer les pertes dans un fichier.

    Args:
        model (torch.nn.Module): Le modèle à entraîner.
        criterion (torch.nn.Module): La fonction de perte à utiliser pour calculer la perte.
        optimizer (torch.optim.Optimizer): L'optimiseur à utiliser pour mettre à jour les poids du modèle.
        trainloader (torch.utils.data.DataLoader): Le chargeur de données pour les données d'entraînement.
        max_iter (int): Le nombre total d'itérations d'entraînement (epochs).
        writer (SummaryWriter): L'écrivain Tensorboard pour enregistrer les statistiques d'entraînement.
        device (torch.device): Le périphérique d'exécution (GPU ou CPU).

    Returns:
        list: Une liste contenant les pertes moyennes à chaque époque.
    """

    model.to(device)  # Transférer le modèle sur le GPU s'il est disponible
    model.train()  # Mettre le modèle en mode d'entraînement

    losses = []  # Liste pour stocker les pertes moyennes à chaque époque

    for epoch in range(max_iter):  # Boucle sur le nombre total d'itérations d'entraînement (epochs)
        running_loss = 0  # Initialiser la perte courante à zéro

        for i, (data, target) in enumerate(trainloader):  # Boucle sur les données d'entraînement
            data, target = data.to(device), target.to(device)  # Transférer les données sur le GPU s'il est disponible
            optimizer.zero_grad()  # Réinitialiser les gradients à zéro
            pred = model(data)  # Passer les données dans le modèle pour obtenir les prédictions
            loss = criterion(pred, target.long())  # Calculer la perte
            loss.backward()  # Calculer les gradients par rétropropagation
            optimizer.step()  # Mettre à jour les poids du modèle
            running_loss += loss.item()  # Ajouter la perte à la perte courante

            # Afficher les statistiques et les enregistrer dans Tensorboard toutes les 1000 itérations
            if (i % 500 == 0):
                print(f"Epoch: {epoch+1} / {max_iter}, Steps: {i+1} / {len(trainloader)}, Loss: {loss.item()}")
                step = epoch * len(trainloader) + i
                writer.add_scalar("Loss", loss.item(), step)  # Enregistrer la perte dans Tensorboard
                writer.add_scalar("Mean/Loss", running_loss/100, step)  # Enregistrer la moyenne de la perte dans Tensorboard
                running_loss = 0  # Réinitialiser la perte courante

        # Calculer la perte moyenne pour l'époque en cours
        epoch_loss = running_loss / len(trainloader)

        # Enregistrer la perte moyenne dans la liste
        losses.append(epoch_loss)

    return losses  # Retourner la liste des pertes moyennes


# Création de la fonction pour l'évaluation du modèle

In [25]:
import torchmetrics as tm
from sklearn.metrics import precision_recall_fscore_support

def evaluate_model(model, criterion, testloader, device, Nb_class):
    """
    Fonction pour évaluer le modèle de détection d'intrusion sur l'ensemble de données de test et
    enregistrer les pertes individuelles et les métriques de performance.

    Args:
        model (torch.nn.Module): Le modèle de détection d'intrusion entraîné.
        criterion (torch.nn.Module): La fonction de perte utilisée pour l'évaluation.
        testloader (torch.utils.data.DataLoader): Le chargeur de données pour l'ensemble de données de test.
        device (torch.device): Le périphérique d'exécution (GPU ou CPU).

    Returns:
        dict: Un dictionnaire contenant les métriques de performance (perte, précision, rappel, F1-score, AUC).
    """

    model.eval()  # Basculer le modèle en mode évaluation

    losses = []  # Liste pour stocker les pertes individuelles
    precision_scores = []
    recall_scores = []
    f1_scores = []
    accuracies = []

    with torch.no_grad():
        accuracy = tm.Accuracy(task='MULTICLASS', num_classes=Nb_class).to(device)  # Instancier la métrique accuracy multiclasse
        
        for i, data in enumerate(testloader):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            losses.append(loss.item())

            pred = outputs.argmax(dim=1)
            
            # Calculer la précision, le rappel et le F1-score à l'aide de sklearn.metrics
            precision, recall, f1_score, _ = precision_recall_fscore_support(
                labels.cpu(), pred.cpu(), average='macro', zero_division=0
            )

            # Appeler les métriques instanciées
            accuracy(pred, labels)

            precision_scores.append(precision)  # Récupérer la valeur via compute()
            recall_scores.append(recall)
            f1_scores.append(f1_score)
            accuracies.append(accuracy.compute())
            

    epoch_loss = sum(losses) / len(losses)
    epoch_precision = sum(precision_scores) / len(precision_scores)
    epoch_recall = sum(recall_scores) / len(recall_scores)
    epoch_f1 = sum(f1_scores) / len(f1_scores)
    epoch_accu = sum(accuracies) / len(accuracies)
    

    # Renvoyer un dictionnaire contenant les métriques
    metrics = {
        "Test set loss": epoch_loss,
        "Accuracy means": epoch_accu,
        "Precision": epoch_precision,
        "Recall": epoch_recall,
        "F1-score": epoch_f1
    }
    return metrics

In [8]:
directory = '/kaggle/input/ids-intrusion-csv/'
# Appel de la fonction pour charger les données
df, nb_file = load_csv_files_progressively(directory)

Fichier utilisé :  02-28-2018.csv


/tmp/ipykernel_34/216919516.py:31: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Fichier utilisé :  02-15-2018.csv
Fichier utilisé :  02-21-2018.csv
Fichier utilisé :  03-02-2018.csv
Fichier utilisé :  02-22-2018.csv
Fichier utilisé :  02-14-2018.csv
Fichier utilisé :  02-23-2018.csv


In [9]:
## Information
print("Le nombre de fichier concatené = ", nb_file, "/10")

print("Shape du dataset charger = ", df.shape)

Le nombre de fichier concatené =  7 /10
Shape du dataset charger =  (6904554, 79)


In [26]:
# Affichage des 5 premières ligne du jeu de donnée
df.head()

,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,443,6,94658,6,7,708,3718,387,0,118.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
1,443,6,206,2,0,0,0,0,0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
2,445,6,165505,3,1,0,0,0,0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
3,443,6,102429,6,7,708,3718,387,0,118.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
4,443,6,167,2,0,0,0,0,0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6904554 entries, 0 to 6904553
Data columns (total 79 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   Dst Port           object
 1   Protocol           object
 2   Flow Duration      object
 3   Tot Fwd Pkts       object
 4   Tot Bwd Pkts       object
 5   TotLen Fwd Pkts    object
 6   TotLen Bwd Pkts    object
 7   Fwd Pkt Len Max    object
 8   Fwd Pkt Len Min    object
 9   Fwd Pkt Len Mean   object
 10  Fwd Pkt Len Std    object
 11  Bwd Pkt Len Max    object
 12  Bwd Pkt Len Min    object
 13  Bwd Pkt Len Mean   object
 14  Bwd Pkt Len Std    object
 15  Flow Byts/s        object
 16  Flow Pkts/s        object
 17  Flow IAT Mean      object
 18  Flow IAT Std       object
 19  Flow IAT Max       object
 20  Flow IAT Min       object
 21  Fwd IAT Tot        object
 22  Fwd IAT Mean       object
 23  Fwd IAT Std        object
 24  Fwd IAT Max        object
 25  Fwd IAT Min        object
 26  Bwd IAT Tot   

**NB :** A cause de certains fichiers notamment:
*     02-28-2018.csv
*     03-01-2018.csv
*     02-16-2018.csv

Pandas n'arrive pas à déterminer les types des colonnes et donc il prend le type par defaut qui est`object`. Donc lors de preprocessing nous avons effectuer une transformation spécifique.


# Partie Principale

## 1. Partie d'obtention des données prêt pour l'entrainement

In [12]:
# Définir le périphérique d'exécution (GPU ou CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Exécution sur {device}")

Exécution sur cuda


In [13]:
# Charger les données CSV et les prétraiter
data_processor = DataProcessor(device)
X, y = data_processor.preprocess_data(df, "Label")
print("Données chargées et prétraitées.")

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print("Données divisées en ensembles d'entraînement et de test.")

# Créer des ensembles de données et des chargeurs de données pour l'entraînement et l'évaluation
data_loader_manager = DataLoaderManager(device=device)
trainset, testset, trainloader, testloader = data_loader_manager.create_datasets_and_loaders(X_train, X_test, y_train, y_test)
in_features, out_features = X_train.shape[1], len(np.unique(y_train.cpu()))
print("Ensembles de données et chargeurs de données créés.")


Données chargées et prétraitées.
Données divisées en ensembles d'entraînement et de test.
Ensembles de données et chargeurs de données créés.


## 2. Initialisation du modèle, de la fonction perte et de l'optimiseur

In [14]:
# Initialiser le modèle de détection d'intrusion
model = IntrusionDetectionModel(in_features, out_features, device)
print("Modèle de détection d'intrusion initialisé.")

# Définir la fonction de perte et l'optimiseur pour l'entraînement
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
print("Critère et optimiseur définis.")

Modèle de détection d'intrusion initialisé.
Critère et optimiseur définis.


## 3. Entrainement du modèle

In [15]:
# Initialiser le writer de Tensorboard pour enregistrer les métriques d'entraînement
writer = SummaryWriter()
print("Writer de Tensorboard initialisé.")

# Entraîner le modèle sur l'ensemble de données d'entraînement
max_iter = 2
train_model(model, criterion, optimizer, trainloader, max_iter, writer, device)
print("Modèle entraîné sur l'ensemble de données d'entraînement.")

Writer de Tensorboard initialisé.
Epoch: 1 / 2, Steps: 1 / 55237, Loss: 2.497056484222412
Epoch: 1 / 2, Steps: 501 / 55237, Loss: 0.5883442163467407
Epoch: 1 / 2, Steps: 1001 / 55237, Loss: 0.6718313694000244
Epoch: 1 / 2, Steps: 1501 / 55237, Loss: 0.38286054134368896
Epoch: 1 / 2, Steps: 2001 / 55237, Loss: 0.19746530055999756
Epoch: 1 / 2, Steps: 2501 / 55237, Loss: 0.22470836341381073
Epoch: 1 / 2, Steps: 3001 / 55237, Loss: 0.13742442429065704
Epoch: 1 / 2, Steps: 3501 / 55237, Loss: 0.11437146365642548
Epoch: 1 / 2, Steps: 4001 / 55237, Loss: 0.2636166512966156
Epoch: 1 / 2, Steps: 4501 / 55237, Loss: 0.11005084961652756
Epoch: 1 / 2, Steps: 5001 / 55237, Loss: 0.09039358794689178
Epoch: 1 / 2, Steps: 5501 / 55237, Loss: 0.0916852056980133
Epoch: 1 / 2, Steps: 6001 / 55237, Loss: 0.1253376603126526
Epoch: 1 / 2, Steps: 6501 / 55237, Loss: 0.05603886768221855
Epoch: 1 / 2, Steps: 7001 / 55237, Loss: 0.15858130156993866
Epoch: 1 / 2, Steps: 7501 / 55237, Loss: 0.08005820214748383
E

## 4. Evaluation du modèle

In [27]:
# Nombre de classes
Nb_class = len(df['Label'].unique())

# Évaluer le modèle sur l'ensemble de données de test et afficher les métriques
metrics = evaluate_model(model, criterion, testloader, device, Nb_class)

print("Métriques d'évaluation:")
for metric_name, value in metrics.items():
    print(f"{metric_name}: {value:.4f}")
print("Modèle évalué sur l'ensemble de données de test.")

# Fermer le writer de Tensorboard
writer.close()
print("Writer de Tensorboard fermé.")

Métriques d'évaluation:
Test set loss: 0.0526
Accuracy means: 0.9884
Precision: 0.9491
Recall: 0.9525
F1-score: 0.9506
Modèle évalué sur l'ensemble de données de test.
Writer de Tensorboard fermé.
